In [2]:
import pandas as pd

In [96]:
audiocaps = {
    "train": pd.read_csv("../data/audioCaps/csvs/train.csv"),
    "valid": pd.read_csv("../data/audioCaps/csvs/valid.csv"),
    "test": pd.read_csv("../data/audioCaps/csvs/test.csv"),
}

for name, split in audiocaps.items():
    print(name, len(split), "unique:",  len(split["youtube_id"].unique()))


train 49838 unique: 49838
valid 2475 unique: 495
test 4875 unique: 975


In [98]:
audiocaps_all = pd.concat(audiocaps.values())
len(audiocaps_all)

57188

In [99]:
audiocaps_all.columns

Index(['audiocap_id', 'youtube_id', 'start_time', 'caption'], dtype='object')

In [100]:
import os
exists = audiocaps_all["youtube_id"].drop_duplicates().apply(lambda x: os.path.exists("../data/audioset_full/audios/train_unbalanced/" + x + ".mp3"))

In [101]:
exists.mean()

0.9107351680049894

In [103]:
exists_2 = audiocaps_all["youtube_id"].drop_duplicates().apply(lambda x: os.path.exists("../data/audioset_full/audios/train_balanced/" + x + ".mp3"))
exists_3 = audiocaps_all["youtube_id"].drop_duplicates().apply(lambda x: os.path.exists("../data/audioset_full/audios/eval/" + x + ".mp3"))

In [76]:
exists.sum() == (exists | exists_2 | exists_3).sum()

True

In [105]:
audioset_small = {
    "train": pd.read_json("../data/audioset_small/annotations/train.jsonl", lines=True),
    "valid": pd.read_json("../data/audioset_small/annotations/valid.jsonl", lines=True),
    "test": pd.read_json("../data/audioset_small/annotations/test.jsonl", lines=True),
}

audioset_small_all = pd.concat(audioset_small.values())

In [108]:
splits = {"train", "valid", "test"}

# CHECK LEAKAGE

for split in splits:
    for other_split in splits - {split}:
        assert set.isdisjoint(set(audioset_small[split]["youtube_id"]), set(audioset_small[other_split]["youtube_id"]))
        assert set.isdisjoint(set(audiocaps[split]["youtube_id"]), set(audiocaps[other_split]["youtube_id"]))
        assert set.isdisjoint(set(audioset_small[split]["youtube_id"]), set(audiocaps[other_split]["youtube_id"]))
